<center><img src="logo.png" /></center>

# <center>Avey: A High-Level Overview</center>

## Table of Contents

* [Introduction](#intro)
* [Cases](#cases)
* [Metrics and Calculations](#metrics)
* [Results](#results)

<p>


## Introduction<a class="anchor" id="intro"></a>
<p>This notebook is a supplement of the <a href="https://www.medrxiv.org/content/10.1101/2022.03.08.22272076v1.full"> paper </a> we are submitting. In this notebook, we share all the cases (after manual cleaning and matching) that we analysed and their results. We also share results from multiple experiments, few of which were discussed in the paper.</p>


#### Load data

In [1]:
EXPERIMENT_TAG = 'avey'
# EXPERIMENT_TAG = 'harvard'

In [2]:
# load data

import json
import pandas as pd
from collections import defaultdict
import math
import numpy as np
import os

def loadData(fileName):
    '''Loading data from result files'''
    with open(f'{fileName}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def normalize(cases):
    '''make all the ddx list of a case of the same length by padding with None'''
    for case in cases.values():
        maxLen = max(len(result) for result in case.values())
        for result in case.values():
            result += [None]*(maxLen-len(result))

        assert len(set(len(result) for result in case.values())) == 1
    
    return cases

def getDataframe(case):
    '''Convert each test case into a dataframe'''
    caseLen = len(next(iter(case.values())))
    if EXPERIMENT_TAG == 'avey':
        gold = ['gold','gold_old']
    else:
        gold = ['gold']
    return pd.DataFrame(
        case,
        columns=[*gold,
        *sorted([key for key in case.keys() if 'gold' not in key and 'doctor' not in key]),
        *sorted([key for key in case.keys() if 'gold' not in key and 'doctor' in key]),
        ],
        index= list(range(1,1+caseLen))
        )


In [3]:
# We need to make all the differentials of the same length to ease comparison
# We pad the lists with None
data = loadData('data/allResults')
if EXPERIMENT_TAG != 'harvard':
    dataOld = loadData('data/allResults_old')

for caseNum in data:
    for app in data[caseNum]:
        # removing empty strings
        data[caseNum][app] = [r for r in data[caseNum][app] if r]

    # adding old results
    if EXPERIMENT_TAG != 'harvard' and caseNum in dataOld:
        for app in dataOld[caseNum]:
            data[caseNum][f"{app}_old"] = [r for r in dataOld[caseNum][app] if r]

normalizedData = normalize(data)
if EXPERIMENT_TAG == 'harvard':
    cases = {int(id): getDataframe(case) for id, case in normalizedData.items() if int(id)>500}
else:
    cases = {int(id): getDataframe(case) for id, case in normalizedData.items() if int(id)<=500}

display(f'We have {len(cases)} cases in the experiment.')

'We have 119 cases in the experiment.'

In [5]:
caseClassificationOld = loadData("data/case-classification_old")
failures_temp = (
    pd.read_excel("data/AppTest-2024-01-30.xlsx")
    .dropna(subset=["invalid_code"])
    .groupby(["medical_app", "invalid_code"])["case_study"]
    .agg(set)
    .reset_index()
).set_index(["medical_app", "invalid_code"]).to_dict('index')

failures = defaultdict(dict)
for (app, err), failedCases in  failures_temp.items():
    failures[app][err] = failedCases['case_study']

for app in failures:
    failures[app]["session failed"] = set()
    for err, failedCases in failures[app].items():
        if err != 'NDX':
            failures[app]["session failed"] |= failedCases

    failures[app]["no disease found"] = failures[app]["NDX"]
    del failures[app]["NDX"]

failures = dict(failures)

if EXPERIMENT_TAG == 'avey':
    for app, data in caseClassificationOld['apps'].items():
        failures[app] = data 
# failures

In [10]:
df = pd.read_excel(
    "data/Results - Apps performance - team copy.xlsx",
    sheet_name=(
        "Harvard cases"
        if EXPERIMENT_TAG == "harvard"
        else "Detailed diseases coverage - 50"
    ),
    index_col=1,
).dropna(subset=["Common vs Less common"])["Common vs Less common"]

df.index = [int(str(i).replace('#','')) for i in df.index]
caseClassification = {
    'common':set(),
    'less common':set(),
    'apps':failures,
}
for case_num, typ in df.items():
    if typ == 'Common':
        caseClassification['common'].add(case_num)
    else:
        caseClassification['less common'].add(case_num)

assert set(cases.keys()).issubset(
    caseClassification["common"] | caseClassification["less common"]
), f'Vignette results have extra ids that are not in classification {set(cases.keys()) - (caseClassification["common"] | caseClassification["less common"])}'

print(f'We have {len(caseClassification["common"])} common diseases and {len(caseClassification["less common"])} less common diseases')
display('App v Error code: Case Count')
pd.DataFrame({app: {err: len(v) for err, v in errs.items()} for app, errs in caseClassification["apps"].items()}).transpose()

We have 272 common diseases and 228 less common diseases


c:\Users\swapn\anaconda3\envs\mlflow\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


'App v Error code: Case Count'

,MAC,OTH,session failed,no disease found,A01,CNA,A16,A18
Avey,3.0,1.0,4.0,29.0,NaN,NaN,NaN,NaN
Avey v2,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
Buoy,8.0,7.0,25.0,4.0,8.0,2.0,NaN,NaN
Healthily,NaN,NaN,65.0,18.0,NaN,5.0,60.0,1.0
K Health,10.0,3.0,96.0,1.0,NaN,19.0,NaN,64.0
Mediktor,1.0,NaN,2.0,41.0,NaN,1.0,NaN,NaN
Symptomate,1.0,NaN,9.0,16.0,NaN,NaN,NaN,8.0
WebMD,NaN,NaN,6.0,1.0,5.0,1.0,NaN,NaN
Avey_old,NaN,NaN,0.0,2.0,NaN,NaN,NaN,NaN
Ada_old,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


## TODO: NEED TO REMOVE DUPLICATE TESTS AND VERIFY THE NUMBER OF ERRORS

<p>

### Let us have a look at all the cases. <a class="anchor" id="cases"></a>
The cases have been labelled common and less common by our doctors. We have also labelled the cases with apps that failed on them.

In [22]:
from IPython.display import display

for caseNum, case in cases.items():
    isCommonString = 'common' if caseNum in caseClassification['common'] else 'less common'
    sessionFailed, noDiseaseFound = ([], [])
    for app, failedCases in caseClassification['apps'].items():
        if caseNum in failedCases['session failed']:
            sessionFailed.append(app)
        elif caseNum in failedCases['no disease found']:
            noDiseaseFound.append(app)

    print(f'Case number {caseNum} ({isCommonString})')
    if sessionFailed:
        print(
            f'Session failed to start for: {", ".join(sessionFailed)}.')
    if noDiseaseFound:
        print(
            f'No diseases were found for: {", ".join(noDiseaseFound)}.')
    display(case)
    case.to_excel(f'individual_case_outputs/{EXPERIMENT_TAG}/{caseNum}.xlsx')
    print('\n'*2)


Case number 1 (common)
No diseases were found for: Babylon_old.


,gold,gold_old,Ada,Ada_old,Avey,Avey v2,Avey_old,Babylon_old,Buoy,Buoy_old,...,Mediktor,Symptomate,WebMD,WebMD_old,doctor_MA,doctor_MA_old,doctor_NJ,doctor_NJ_old,doctor_TH,doctor_TH_old
1,acute bronchitis,acute bronchitis,acute bronchitis,acute bronchitis,acute bronchitis,acute bronchitis,acute bronchitis,None,acute bronchitis,acute bronchitis,...,asthma,acute bronchitis,pneumonia,asthma,gastroesophageal reflux,gastroesophageal reflux,asthma,reactive airway disease with superimposed infe...,post infectious cough,post infectious cough
2,common cold,common cold,common cold,asthma,covid 19,bacterial tracheitis,common cold,None,benign cough,asthma,...,acute bronchitis,tonsillopharyngitis,influenza,acute bronchitis,post nasal drip,post nasal drip,None,None,acute bronchitis,acute bronchitis
3,influenza,influenza,acute laryngitis,whooping cough,common cold,common cold,pneumonia,None,influenza,benign cough,...,hyperresponsive bronchi,common cold,asthma,coronavirus disease 2019,acute bronchitis,acute bronchitis,None,None,None,None
4,asthma,asthma,covid 19,pneumonia,cystic fibrosis,foreign body aspiration,pulmonary edema,None,None,None,...,None,acute laryngitis,common cold,influenza,None,None,None,None,None,None
5,None,None,pneumonia,mycoplasma pneumonia,pneumonia,acute laryngitis,laryngitis,None,None,None,...,None,acute sinusitis,chronic sinusitis,chronic sinusitis,None,None,None,None,None,None
6,None,None,None,None,None,None,tonsillopharingitis,None,None,None,...,None,None,whooping cough,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,acute bronchitis,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,lung cancer,None,None,None,None,None,None,None





Case number 2 (common)
No diseases were found for: Babylon_old, Buoy_old.


,gold,gold_old,Ada,Ada_old,Avey,Avey v2,Avey_old,Babylon_old,Buoy,Buoy_old,...,Mediktor,Symptomate,WebMD,WebMD_old,doctor_MA,doctor_MA_old,doctor_NJ,doctor_NJ_old,doctor_TH,doctor_TH_old
1,unstable angina,unstable angina,unstable angina,unstable angina,unstable angina,unstable angina,unstable angina,None,unstable angina,None,...,myocardial infarction,unstable angina,myocardial infarction,myocardial infarction,stable angina,stable angina,unstable angina,unstable angina,myocardial infarction,myocardial infarction
2,myocardial infarction,myocardial infarction,myocardial infarction,collapsed lung,myocardial infarction,pulmonary embolism,myocardial infarction,None,atypical chest pain,None,...,myocardial infarction,myocardial infarction,unstable angina,broken ribs,unstable angina,unstable angina,myocardial infarction,myocardial infarction,unstable angina,unstable angina
3,stable angina,stable angina,acute biventricular heart failure,myocardial infarction,stable angina,myocardial infarction,stable angina,None,heart valve issues,None,...,unstable angina,None,pulmonary embolism,costochondritis,myocardial infarction,myocardial infarction,None,None,heart failure,heart failure
4,pericardial effusion,pericardial effusion,acute aortic regurgitation,chronic obstructive pulmonary disease,aortic dissection,pericardial effusion,varient angina,None,None,None,...,None,None,congestive heart failure,heartburn/gerd,None,None,None,None,None,None
5,None,None,obstructive hypertrophic cardiomyopathy,stable angina,costochondritis,aortic dissection,None,None,None,None,...,None,None,atrial fibrillation,unstable angina,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,...,None,None,heart failure with preserved ejection fraction,pulmonary embolism,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,takotsubo cardiomyopathy,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,heart burn gerd,None,None,None,None,None,None,None





Case number 3 (less common)
No diseases were found for: Babylon_old, Buoy_old.


...

,gold,gold_old,Ada,Ada_old,Avey,Avey v2,Avey_old,Babylon_old,Buoy,Buoy_old,...,Mediktor,Symptomate,WebMD,WebMD_old,doctor_MA,doctor_MA_old,doctor_NJ,doctor_NJ_old,doctor_TH,doctor_TH_old
1,sickle cell anemia,sickle cell anemia,familial mediterranean fever,familial mediterranean fever,rheumatoid arthritis,rheumatoid arthritis,osteomyelitis,None,None,None,...,septic arthritis,None,None,None,sickle cell anemia,sickle cell anemia,sickle cell anemia,sickle cell anemia,sickle cell anemia,sickle cell anemia
2,septic arthritis,septic arthritis,septic arthritis,systemic onset juvenile idiopathic arthritis,septic arthritis,pseudogout,deep vein thrombosis,None,None,None,...,gout,None,None,None,None,None,None,None,None,None
3,None,None,sickle cell anemia,None,pseudogout,septic arthritis,nephrotic syndrome,None,None,None,...,sepsis,None,None,None,None,None,None,None,None,None
4,None,None,rheumatic fever,None,lyme disease,lyme disease,acute prostatitis,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,None,None,meningococcemia,None,polymyalgia rheumatica,systemic lupus erythematosus,amyloidosis,None,None,None,...,None,None,None,None,None,None,None,None,None,None





Case number 497 (common)
No diseases were found for: Babylon_old.


,gold,gold_old,Ada,Ada_old,Avey,Avey v2,Avey_old,Babylon_old,Buoy,Buoy_old,...,Mediktor,Symptomate,WebMD,WebMD_old,doctor_MA,doctor_MA_old,doctor_NJ,doctor_NJ_old,doctor_TH,doctor_TH_old
1,mixed urinary incontinence,mixed urinary incontinence,prostate cancer,mixed urinary incontinence,benign prostatic hyperplasia,mixed urinary incontinence,mixed urinary incontinence,None,benign prostatic hyperplasia,benign prostatic hypertrophy (bph),...,urge urinary incontinence,stress urinary incontinence,benign prostatic hyperplasia,urge urinary incontinence,mixed urinary incontinence,mixed urinary incontinence,stress urinary incontinence,stress urinary incontinence,uti,uti
2,stress urinary incontinence,stress urinary incontinence,mixed urinary incontinence,stress urinary incontinence,urge urinary incontinence,urge urinary incontinence,overactive bladder,None,diabetes mellitus,None,...,urinary tract infection,benign prostatic hyperplasia,urge urinary incontinence,prostate gland enlargement,None,None,None,None,urge urinary incontinence,urge urinary incontinence
3,urge urinary incontinence,urge urinary incontinence,benign prostatic hyperplasia,lumbar spinal stenosis,bladder cancer,None,atrophic urethritis,None,acute prostatitis,None,...,anxiety disorder,None,prostate cancer,interstitial cystitis,None,None,None,None,None,None
4,None,None,urge urinary incontinence,None,None,None,urogenital fistula,None,None,None,...,None,None,deep venous thrombosis,prostate cancer,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,...,None,None,cerebral stroke,congenial heart disease,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,...,None,None,brain aneurysm,parkinson disease,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,heart failure,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,lymphoma,None,None,None,None,None,None,None





Case number 499 (less common)
No diseases were found for: Avey, Babylon_old.


,gold,gold_old,Ada,Ada_old,Avey,Avey v2,Avey_old,Babylon_old,Buoy,Buoy_old,...,Mediktor,Symptomate,WebMD,WebMD_old,doctor_MA,doctor_MA_old,doctor_NJ,doctor_NJ_old,doctor_TH,doctor_TH_old
1,lymphedema,lymphedema,lymphedema,lymphedema,None,chronic venous insufficiency,lymphedema,None,cushing syndrome,atrial fibrillation,...,deep venous thrombosis,chronic venous insufficiency,varicose veins,deep venous thrombosis,insect bite,insect bite,deep venous thrombosis,deep venous thrombosis,lymphedema,elephantiasis / filariasis
2,deep venous thrombosis,deep venous thrombosis,deep venous thrombosis,venous insufficiency,None,lymphedema,chronic venous insufficiency,None,atrial fibrillation,None,...,heart failure,deep venous thrombosis,nephrotic syndrome,thrombophlebitis,deep venous thrombosis,deep venous thrombosis,baker's cyst rupture,baker's cyst rupture,lymphedema,lymphedema
3,None,None,chronic lymphatic filariasis,None,None,cellulitis,nephrotic syndrome,None,None,None,...,bursitis,None,heart failure,cellulitis,lymphedema,lymphedema,None,None,None,None
4,None,None,None,None,None,deep venous thrombosis,liver cirrhosis,None,None,None,...,None,None,deep venous thrombosis,pulmonary embolism,fluids stasis,fluids stasis,None,None,None,None
5,None,None,None,None,None,superficial thrombophlebitis,nephritic syndrome,None,None,None,...,None,None,sprained ankle,baker's cyst,None,None,None,None,None,None
6,None,None,None,None,None,None,acute renal failure,None,None,None,...,None,None,broken ankle,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,obesity,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,obstructive sleep apnea,None,None,None,None,None,None,None


<p>

## Let us define the metrics now. <a class="anchor" id="metrics"></a>

### Terms used
- TP: True positive (correct disease retrieved)
- TN: True negative (wrong disease **not** retrieved)
- FP: False positive (wrong disease retrieved)
- FN: False negative (correct disease **not** retrieved)
- gold standard - the correct list of diseases as determined by collective intelligence of doctors

### Precision
Precision helps us understand how exact our results are. It gives us an intuition about how many wrong diseases (false positives) are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the complete list retrieved*.
$$precision = \frac{TP}{TP + FP} = \frac{TP}{\text{length of differential list}}$$

### Recall
Recall is a measure of how many of the correct diseases are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the gold standard list*.
$$recall = \frac{TP}{TP + FN} = \frac{TP}{\text{length of the gold standard}}$$

### F1 Score
F1 score is the weighted harmonic mean of *precision* and *recall*. It is a metric that combines *precision* and *recall* and gives us 1 score for easier comparison.

Suppose $\beta$ defines how important is $recall$ to $precision$ then,
$$fscore_{\beta} = (1 + \beta^2)\frac{precision \cdot recall}{(\beta^2 \cdot precision) + recall}$$
Substituting $\beta = 1$,
$$fscore_{1} = \frac{2 \cdot precision \cdot recall}{ precision + recall}$$

### NDCG
NDCG or Normalized Discounted Cumulative Gain is measure of how accurate the ranking is. In our calculations, we use
$$DCG = \sum_{i=1}^n\frac{2^{relevance_i}-1}{log_2(i+1)}$$
where $n$ is the number of differentials in the returned list and  
$relevance_i = |gold standard| - rank_{gold\ standard}(ddx[i])$ if $ddx[i]$ is present, 0 otherwise.

$$NDCG = \frac{DCG_{ddx}}{DCG_{gold\ standard}}$$

### M Score
M Score determines where the top disease (gold standard) appears in the returned differential.
$$M_i = \text{gold standard[0]} \in \text{ddx[:i]}$$

### Position
Shows the position of the gold standard[0] in the returned differential. 

### Length
$$length = \frac{|ddx|}{|gold\ standard|}$$
<br>
<br>
<br>
<br>


In [15]:
import math

def getPrecision(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp == 0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()


def getFScore(precision: float, recall: float, beta: float = 1) -> float:
    return math.nan if precision+recall == 0 else \
        (1+beta**2)*precision*recall/(precision*(beta**2)+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score: float, index: int) -> float:
        '''The index is 1 based'''
        return (math.pow(2, score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i], i+1) for i in range(len(scores)))

    candidateRelevance = []
    goldStandard = list(goldStandard)
    for index, disease in enumerate(candidate):
        if disease is not None and disease in goldStandard:
            candidateRelevance.append(
                discount(scores[goldStandard.index(disease)], index+1))
        else:
            candidateRelevance.append(0)

    return sum(candidateRelevance)/maxDCG


def getMScore(goldStandard: pd.Series, candidate: pd.Series, m=1) -> bool:
    return goldStandard.values[0] in candidate.values[:m]


def getPosition(goldStandard: pd.Series, candidate: pd.Series) -> bool:
    return math.nan if goldStandard.values[0] not in candidate.values else\
        1 + list(candidate.values).index(goldStandard.values[0])


def getLength(goldStandard: pd.Series, candidate: pd.Series) -> int:
    return math.nan if candidate.count() == 0 else \
        candidate.count()/goldStandard.count()


def getScoresCase(id,case: pd.DataFrame) -> pd.DataFrame:
    gold = case['gold']
    if EXPERIMENT_TAG == 'avey':
        gold_old = case['gold_old']
    else:
        gold_old = gold

    scores = [
        [getPrecision(gold_old if '_old' in col else gold, case[col]) for col in case.columns if "gold" not in col],
        [getRecall(gold_old if '_old' in col else gold, case[col]) for col in case.columns if "gold" not in col],
    ]

    scores.append(
        [getFScore(prec, rec, beta=1) for prec, rec in zip(scores[0], scores[1])]
    )
    scores.append(
        [getFScore(prec, rec, beta=2) for prec, rec in zip(scores[0], scores[1])]
    )

    # relevance for a list of 4 is 4, 3, 2, 1
    # relevance for a list of 2 is 2, 1
    scores.append(
        [
            getNDCG(
                gold_old if "_old" in col else gold,
                case[col],
                list(range(gold.count(), 0, -1)),
            )
            for col in case.columns
            if "gold" not in col
        ]
    )

    for m in range(1, 6, 2):
        scores.append(
            [
                getMScore(gold_old if "_old" in col else gold, case[col], m)
                for col in case.columns
                if "gold" not in col
            ]
        )

    try:
        scores.append(
            [
                getPosition(gold_old if "_old" in col else gold, case[col])
                for col in case.columns
                if "gold" not in col
            ]
        )
    except Exception as e:
        print(id,flush=True)
        raise e

    scores.append(
        [
            getLength(gold_old if "_old" in col else gold, case[col])
            for col in case.columns
            if "gold" not in col
        ]
    )

    return pd.DataFrame(
        scores,
        columns=[col for col in case.columns if "gold" not in col],
        index=[
            "precision",
            "recall",
            "f1-score",
            "f2-score",
            "NDCG",
            "M1",
            "M3",
            "M5",
            "position",
            "length (x of gs)",
        ],
    )

scores = {id:getScoresCase(id,case) for id, case in cases.items()}

In [16]:
for id, score in scores.items():
    score.to_excel(f'individual_results/{EXPERIMENT_TAG}/{id}.xlsx')

In [17]:
print(len(scores))

119


Let us define the experiment now. We will pick which cases to compute startistics for.

In [26]:
from enum import Enum 

class DiseaseType(Enum):
    ALL = 'all'
    COMMON = 'common-diseases'
    UNCOMMON = 'less-common-diseases'

class FailureType(Enum):
    ANY = 'any-error'
    NO_DDX = 'no-disease-found'
    SES_FAIL = 'session-failed'
    NONE = 'None'

In [27]:
from collections import defaultdict

experiments = {}

def getExpName(disease_type:DiseaseType,failure_type:FailureType=FailureType.NONE,app='NA'):
    return f"disease_type={disease_type.value} failure-type-ignored={failure_type.value} for app={app}"

def addExperiment(disease_type, casesToConsider):
    # add experiment to ignore no case
    experiments[getExpName(disease_type)] = set(casesToConsider)

    caseClassificationApps = {}
    for app, failure_type in caseClassification["apps"].items():
        caseClassificationApps[app] = {
            FailureType.NO_DDX: failure_type["no disease found"],
            FailureType.SES_FAIL: failure_type["session failed"],
            FailureType.ANY: set(failure_type["no disease found"])
            | set(failure_type["session failed"]),
        }

    for app, classifications in caseClassificationApps.items():
        for failure_type, caseNums in classifications.items():

            # experiment to ignore the cases for the particular app
            experiments[getExpName(disease_type,failure_type,app)] = set(casesToConsider) - set(caseNums)

            # experiment to ignore the cases for all the apps
            if getExpName(disease_type,failure_type,'any') in experiments:
                experiments[getExpName(disease_type,failure_type,'any')] = (
                    experiments[getExpName(disease_type,failure_type,'any')]
                    - set(caseNums)
                )
            else:
                experiments[getExpName(disease_type,failure_type,'any')] = experiments[getExpName(disease_type,failure_type,app)]


addExperiment(DiseaseType.COMMON, caseClassification["common"])
addExperiment(DiseaseType.UNCOMMON, caseClassification["less common"])
addExperiment(
    DiseaseType.ALL,
    set(caseClassification["less common"]) | set(caseClassification["common"]),
)

display("The experiments we are going to conduct are:")
dict(enumerate(sorted(list(experiments.keys()))))

'The experiments we are going to conduct are:'

{0: 'disease_type=all failure-type-ignored=None for app=NA',
 1: 'disease_type=all failure-type-ignored=any-error for app=Ada_old',
 2: 'disease_type=all failure-type-ignored=any-error for app=Avey',
 3: 'disease_type=all failure-type-ignored=any-error for app=Avey v2',
 4: 'disease_type=all failure-type-ignored=any-error for app=Avey_old',
 5: 'disease_type=all failure-type-ignored=any-error for app=Babylon_old',
 6: 'disease_type=all failure-type-ignored=any-error for app=Buoy',
 7: 'disease_type=all failure-type-ignored=any-error for app=Buoy_old',
 8: 'disease_type=all failure-type-ignored=any-error for app=Healthily',
 9: 'disease_type=all failure-type-ignored=any-error for app=K Health',
 10: 'disease_type=all failure-type-ignored=any-error for app=K health_old',
 11: 'disease_type=all failure-type-ignored=any-error for app=Mediktor',
 12: 'disease_type=all failure-type-ignored=any-error for app=Symptomate',
 13: 'disease_type=all failure-type-ignored=any-error for app=WebMD',
 1

<br>
<br>

### Results <a class="anchor" id="results"></a>

In [28]:
#shows the number of cases where Avey failed M1
#Do not run this after Avey v2 is replaces Avey

count = 0
for id, score in scores.items():
    if not score.loc["M1","Avey v2"]:
        print(id)
        count += 1 

count

21
22
24
43
65
66
68
75
83
84
104
118
149
159
160
182
187
195
206
207
216
225
229
233
246
251
261
266
271
290
299
300
308
311
326
334
336
344
354
359
366
393
394
411
414
416
420
423
438
445
450
461
472
474
489
490
491
493
499


59

In [29]:
def getStats(scores, row: str, col: str):
    values = []
    for score in scores.values():
        try:
            if not math.isnan(score.loc[row, col]):
                values.append(score.loc[row, col])
        except KeyError as e:
            if '_old' not in col:
                display(score,row,col)
                raise e
    if not values:
        return {
            (col, "average"): 0,
            (col, "variance"): 0,
            (col, "std Dev"): 0,
        }
    
    average = sum(values)/len(values)
    variance = sum((value-average)**2 for value in values)/len(values)
    stdDev = variance**0.5
    return {
        (col,"average"): round(average, 3), 
        (col,"variance"):round(variance, 3), 
        (col,"std Dev"):round(stdDev, 3),
        }


results = {}
for label, casesToConsider in experiments.items():
    # selectedScores = {id: score for id,
    #                   score in scores.items()}
    selectedScores = {id: score for id,
                      score in scores.items() if int(id) in casesToConsider}
    
    columns = next(iter(scores.values())).columns
    index = [
            "precision",
            "recall",
            "f1-score",
            "f2-score",
            "NDCG",
            "M1",
            "M3",
            "M5",
            "position",
            "length (x of gs)",
    ]

    metrics = defaultdict(list)
    for row in index:
        for stat in [getStats(selectedScores, row, col) for col in columns]:
            for key,val in stat.items():
                metrics[key].append(val)

    averageScores = pd.DataFrame(metrics, index=[f"stats_for_{x}" for x in index])

    # ignore nan for recall
    for col in columns:
        p = averageScores.loc["stats_for_precision", (col,'average')]
        r = averageScores.loc["stats_for_recall", (col, "average")]
        averageScores.loc["stats_for_f1-score", (col, "average")] = round(getFScore(p, r, 1), 3)
        averageScores.loc["stats_for_f2-score", (col, "average")] = round(getFScore(p, r, 2), 3)

        for key in ['variance','std Dev']:
            averageScores.loc["stats_for_f1-score", (col, key)] = np.nan
            averageScores.loc["stats_for_f2-score", (col, key)] = np.nan

    def calcStats(doctorResults):
        def extractItem(data, col='average'):
            return data[col]

        sum = pd.Series(extractItem(doctorResults[0]))
        for data in doctorResults[1:]:
            sum += extractItem(data)
        average = (sum/len(doctorResults)).round(3)
        return average

    doctorResultsAverage = calcStats(
        [
            averageScores.loc[:, "doctor_MA"] ,
            averageScores.loc[:, "doctor_NJ"] ,
            averageScores.loc[:, "doctor_TH"]
        ]
    )

    if EXPERIMENT_TAG != 'harvard':
        doctorResultsAverageOld = calcStats(
            [
                averageScores.loc[:, "doctor_MA_old"] ,
                averageScores.loc[:, "doctor_NJ_old"] ,
                averageScores.loc[:, "doctor_TH_old"]
            ]
        )
        averageScores["average_doctor_old"] = doctorResultsAverageOld

    # averageScores.insert(
    #     loc=8, column="average_doctor",
    #     value=doctorResultsAverage,
    # )

    averageScores["average_doctor"] = doctorResultsAverage

    results[label] = averageScores

In [30]:
def displayResults(results,printNumCases=True):
    for label, result in results.items():
        if printNumCases:
            print(f'Results for experiment `{label}`, which has {len(set(experiments[label]) & set(scores.keys()))} cases, is')
        else:
            print(f'Results for experiment `{label}` is')
        display(result)
        result.to_excel(f'stats/{EXPERIMENT_TAG}/{label}.xlsx')

displayResults({key:val for key, val in results.items()})

Results for experiment `disease_type=common-diseases failure-type-ignored=None for app=NA`, which has 56 cases, is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.471    0.079   0.282   0.478    0.052   0.229   
stats_for_recall             0.555    0.086   0.293   0.599    0.059   0.242   
stats_for_f1-score           0.510      NaN     NaN   0.532      NaN     NaN   
stats_for_f2-score           0.536      NaN     NaN   0.570      NaN     NaN   
stats_for_NDCG               0.616    0.145   0.381   0.706    0.070   0.265   
stats_for_M1                 0.429    0.245   0.495   0.674    0.220   0.469   
stats_for_M3                 0.732    0.196   0.443   0.814    0.151   0.389   
stats_for_M5                 0.750    0.188   0.433   0.860    0.120   0.347   
stats_for_position           1.619    0.760   0.872   1.405    0.836   0.914   
stats_for_length (x of gs)   1.324    0.288   0.537   1.405    0.292   0.540   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.393    0.069   0.262   0.459  ...   
stats_for_recall             0.567    0.118   0.344   0.607  ...   
stats_for_f1-score           0.464      NaN     NaN   0.523  ...   
stats_for_f2-score           0.521      NaN     NaN   0.570  ...   
stats_for_NDCG               0.547    0.113   0.337   0.658  ...   
stats_for_M1                 0.304    0.211   0.460   0.554  ...   
stats_for_M3                 0.643    0.230   0.479   0.768  ...   
stats_for_M5                 0.696    0.211   0.460   0.786  ...   
stats_for_position           1.744    0.703   0.839   1.386  ...   
stats_for_length (x of gs)   1.553    0.367   0.606   1.508  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.141   0.375     0.801    0.069   0.262   
stats_for_recall                   0.050   0.223     0.512    0.054   0.233   
stats_for_f1-score                   NaN     NaN     0.625      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.552      NaN     NaN   
stats_for_NDCG                     0.108   0.329     0.703    0.074   0.272   
stats_for_M1                       0.239   0.489     0.696    0.211   0.460   
stats_for_M3                       0.220   0.469     0.839    0.135   0.367   
stats_for_M5                       0.220   0.469     0.839    0.135   0.367   
stats_for_position                 0.188   0.433     1.170    0.141   0.376   
stats_for_length (x of gs)         0.050   0.224     0.715    0.142   0.377   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.756    0.064   0.253              0.688   
stats_for_recall                   0.521    0.047   0.216              0.433   
stats_for_f1-score                 0.617      NaN     NaN              0.530   
stats_for_f2-score                 0.556      NaN     NaN              0.467   
stats_for_NDCG                     0.675    0.061   0.247              0.590   
stats_for_M1                       0.651    0.227   0.477              0.581   
stats_for_M3                       0.814    0.151   0.389              0.729   
stats_for_M5                       0.814    0.151   0.389              0.729   
stats_for_position                 1.229    0.233   0.483              1.256   
stats_for_length (x of gs)         0.776    0.164   0.404              0.720   

                           average_doctor  
                                           
stats_for_precision                 0.765  
stats_for_recall                    0.456  
stats_for_f1-score                  0.569  
stats_for_f2-score                  0.495  
stats_for_NDCG                      0.625  
stats_for_M1                       

Results for experiment `disease_type=common-diseases failure-type-ignored=no-disease-found for app=Avey`, which has 54 cases, is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.472    0.080   0.283   0.486    0.053   0.230   
stats_for_recall             0.548    0.085   0.292   0.592    0.057   0.239   
stats_for_f1-score           0.507      NaN     NaN   0.534      NaN     NaN   
stats_for_f2-score           0.531      NaN     NaN   0.567      NaN     NaN   
stats_for_NDCG               0.622    0.145   0.381   0.696    0.071   0.267   
stats_for_M1                 0.444    0.247   0.497   0.659    0.225   0.474   
stats_for_M3                 0.741    0.192   0.438   0.805    0.157   0.396   
stats_for_M5                 0.759    0.183   0.428   0.854    0.125   0.353   
stats_for_position           1.610    0.775   0.880   1.429    0.873   0.935   
stats_for_length (x of gs)   1.299    0.272   0.521   1.352    0.245   0.495   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.408    0.065   0.255   0.468  ...   
stats_for_recall             0.588    0.110   0.332   0.610  ...   
stats_for_f1-score           0.482      NaN     NaN   0.530  ...   
stats_for_f2-score           0.540      NaN     NaN   0.575  ...   
stats_for_NDCG               0.567    0.106   0.326   0.667  ...   
stats_for_M1                 0.315    0.216   0.464   0.574  ...   
stats_for_M3                 0.667    0.222   0.471   0.778  ...   
stats_for_M5                 0.722    0.201   0.448   0.796  ...   
stats_for_position           1.744    0.703   0.839   1.372  ...   
stats_for_length (x of gs)   1.553    0.367   0.606   1.489  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.144   0.379     0.793    0.070   0.264   
stats_for_recall                   0.042   0.205     0.503    0.052   0.228   
stats_for_f1-score                   NaN     NaN     0.616      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.543      NaN     NaN   
stats_for_NDCG                     0.107   0.327     0.696    0.075   0.273   
stats_for_M1                       0.243   0.493     0.685    0.216   0.464   
stats_for_M3                       0.225   0.474     0.833    0.139   0.373   
stats_for_M5                       0.225   0.474     0.833    0.139   0.373   
stats_for_position                 0.200   0.448     1.178    0.146   0.382   
stats_for_length (x of gs)         0.047   0.217     0.714    0.145   0.381   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.744    0.064   0.253              0.678   
stats_for_recall                   0.510    0.043   0.208              0.422   
stats_for_f1-score                 0.605      NaN     NaN              0.517   
stats_for_f2-score                 0.544      NaN     NaN              0.455   
stats_for_NDCG                     0.663    0.060   0.246              0.576   
stats_for_M1                       0.634    0.232   0.482              0.561   
stats_for_M3                       0.805    0.157   0.396              0.716   
stats_for_M5                       0.805    0.157   0.396              0.716   
stats_for_position                 1.242    0.244   0.494              1.273   
stats_for_length (x of gs)         0.778    0.168   0.410              0.714   

                           average_doctor  
                                           
stats_for_precision                 0.760  
stats_for_recall                    0.448  
stats_for_f1-score                  0.561  
stats_for_f2-score                  0.487  
stats_for_NDCG                      0.616  
stats_for_M1                       

Results for experiment `disease_type=common-diseases failure-type-ignored=no-disease-found for app=any`, which has 15 cases, is


...

Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.434    0.075   0.273   0.425    0.072   0.268   
stats_for_recall             0.574    0.101   0.318   0.539    0.086   0.294   
stats_for_f1-score           0.494      NaN     NaN   0.475      NaN     NaN   
stats_for_f2-score           0.539      NaN     NaN   0.512      NaN     NaN   
stats_for_NDCG               0.577    0.131   0.361   0.602    0.112   0.335   
stats_for_M1                 0.381    0.236   0.486   0.548    0.248   0.498   
stats_for_M3                 0.653    0.227   0.476   0.677    0.219   0.467   
stats_for_M5                 0.703    0.209   0.457   0.710    0.206   0.454   
stats_for_position           1.747    1.056   1.028   1.409    0.757   0.870   
stats_for_length (x of gs)   1.495    0.344   0.586   1.563    0.650   0.806   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.341    0.067   0.259   0.412  ...   
stats_for_recall             0.532    0.134   0.365   0.631  ...   
stats_for_f1-score           0.416      NaN     NaN   0.499  ...   
stats_for_f2-score           0.478      NaN     NaN   0.570  ...   
stats_for_NDCG               0.464    0.118   0.343   0.628  ...   
stats_for_M1                 0.212    0.167   0.409   0.508  ...   
stats_for_M3                 0.542    0.248   0.498   0.720  ...   
stats_for_M5                 0.619    0.236   0.486   0.737  ...   
stats_for_position           2.055    1.257   1.121   1.414  ...   
stats_for_length (x of gs)   1.753    0.471   0.686   1.743  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.142   0.376     0.751    0.081   0.284   
stats_for_recall                   0.051   0.226     0.540    0.066   0.256   
stats_for_f1-score                   NaN     NaN     0.628      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.572      NaN     NaN   
stats_for_NDCG                     0.107   0.326     0.714    0.073   0.270   
stats_for_M1                       0.240   0.489     0.686    0.215   0.464   
stats_for_M3                       0.226   0.475     0.856    0.123   0.351   
stats_for_M5                       0.226   0.475     0.856    0.123   0.351   
stats_for_position                 0.121   0.349     1.248    0.285   0.534   
stats_for_length (x of gs)         0.071   0.267     0.818    0.191   0.438   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.716    0.080   0.282              0.667   
stats_for_recall                   0.549    0.064   0.253              0.445   
stats_for_f1-score                 0.621      NaN     NaN              0.531   
stats_for_f2-score                 0.576      NaN     NaN              0.476   
stats_for_NDCG                     0.689    0.067   0.259              0.580   
stats_for_M1                       0.645    0.229   0.478              0.566   
stats_for_M3                       0.839    0.135   0.368              0.692   
stats_for_M5                       0.839    0.135   0.368              0.692   
stats_for_position                 1.295    0.336   0.580              1.236   
stats_for_length (x of gs)         0.896    0.283   0.532              0.784   

                           average_doctor  
                                           
stats_for_precision                 0.708  
stats_for_recall                    0.455  
stats_for_f1-score                  0.551  
stats_for_f2-score                  0.489  
stats_for_NDCG                      0.603  
stats_for_M1                       

Results for experiment `disease_type=all failure-type-ignored=session-failed for app=WebMD_old`, which has 116 cases, is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.425    0.073   0.270   0.427    0.071   0.267   
stats_for_recall             0.565    0.100   0.316   0.543    0.085   0.291   
stats_for_f1-score           0.485      NaN     NaN   0.478      NaN     NaN   
stats_for_f2-score           0.530      NaN     NaN   0.515      NaN     NaN   
stats_for_NDCG               0.569    0.133   0.365   0.607    0.110   0.332   
stats_for_M1                 0.379    0.235   0.485   0.549    0.248   0.498   
stats_for_M3                 0.638    0.231   0.481   0.681    0.217   0.466   
stats_for_M5                 0.690    0.214   0.463   0.714    0.204   0.452   
stats_for_position           1.738    1.069   1.034   1.415    0.766   0.875   
stats_for_length (x of gs)   1.503    0.346   0.588   1.580    0.664   0.815   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.337    0.064   0.254   0.408  ...   
stats_for_recall             0.528    0.134   0.366   0.629  ...   
stats_for_f1-score           0.411      NaN     NaN   0.495  ...   
stats_for_f2-score           0.474      NaN     NaN   0.568  ...   
stats_for_NDCG               0.464    0.118   0.343   0.631  ...   
stats_for_M1                 0.216    0.169   0.411   0.517  ...   
stats_for_M3                 0.543    0.248   0.498   0.724  ...   
stats_for_M5                 0.621    0.235   0.485   0.741  ...   
stats_for_position           2.056    1.275   1.129   1.407  ...   
stats_for_length (x of gs)   1.753    0.469   0.685   1.743  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.143   0.378     0.747    0.081   0.285   
stats_for_recall                   0.051   0.227     0.542    0.066   0.258   
stats_for_f1-score                   NaN     NaN     0.628      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.573      NaN     NaN   
stats_for_NDCG                     0.107   0.328     0.716    0.073   0.269   
stats_for_M1                       0.241   0.491     0.690    0.214   0.463   
stats_for_M3                       0.228   0.477     0.862    0.119   0.345   
stats_for_M5                       0.228   0.477     0.862    0.119   0.345   
stats_for_position                 0.125   0.354     1.250    0.287   0.536   
stats_for_length (x of gs)         0.073   0.270     0.825    0.192   0.438   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.710    0.080   0.282              0.662   
stats_for_recall                   0.552    0.065   0.255              0.445   
stats_for_f1-score                 0.621      NaN     NaN              0.529   
stats_for_f2-score                 0.578      NaN     NaN              0.474   
stats_for_NDCG                     0.692    0.067   0.258              0.577   
stats_for_M1                       0.648    0.228   0.477              0.560   
stats_for_M3                       0.846    0.130   0.361              0.688   
stats_for_M5                       0.846    0.130   0.361              0.688   
stats_for_position                 1.299    0.339   0.583              1.243   
stats_for_length (x of gs)         0.906    0.284   0.533              0.790   

                           average_doctor  
                                           
stats_for_precision                 0.704  
stats_for_recall                    0.455  
stats_for_f1-score                  0.549  
stats_for_f2-score                  0.488  
stats_for_NDCG                      0.601  
stats_for_M1                       

Results for experiment `disease_type=all failure-type-ignored=any-error for app=WebMD_old`, which has 115 cases, is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.427    0.073   0.270   0.429    0.071   0.267   
stats_for_recall             0.566    0.101   0.317   0.544    0.086   0.293   
stats_for_f1-score           0.487      NaN     NaN   0.480      NaN     NaN   
stats_for_f2-score           0.531      NaN     NaN   0.516      NaN     NaN   
stats_for_NDCG               0.573    0.132   0.364   0.604    0.111   0.333   
stats_for_M1                 0.383    0.236   0.486   0.544    0.248   0.498   
stats_for_M3                 0.643    0.229   0.479   0.678    0.218   0.467   
stats_for_M5                 0.696    0.212   0.460   0.711    0.205   0.453   
stats_for_position           1.738    1.069   1.034   1.422    0.775   0.880   
stats_for_length (x of gs)   1.495    0.340   0.583   1.570    0.662   0.814   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.340    0.064   0.253   0.412  ...   
stats_for_recall             0.533    0.133   0.364   0.634  ...   
stats_for_f1-score           0.415      NaN     NaN   0.499  ...   
stats_for_f2-score           0.479      NaN     NaN   0.572  ...   
stats_for_NDCG               0.468    0.117   0.342   0.636  ...   
stats_for_M1                 0.217    0.170   0.412   0.522  ...   
stats_for_M3                 0.548    0.248   0.498   0.730  ...   
stats_for_M5                 0.626    0.234   0.484   0.748  ...   
stats_for_position           2.056    1.275   1.129   1.407  ...   
stats_for_length (x of gs)   1.753    0.469   0.685   1.743  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.144   0.379     0.745    0.081   0.285   
stats_for_recall                   0.052   0.228     0.543    0.067   0.259   
stats_for_f1-score                   NaN     NaN     0.628      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.574      NaN     NaN   
stats_for_NDCG                     0.108   0.328     0.715    0.073   0.270   
stats_for_M1                       0.242   0.492     0.687    0.215   0.464   
stats_for_M3                       0.229   0.479     0.861    0.120   0.346   
stats_for_M5                       0.229   0.479     0.861    0.120   0.346   
stats_for_position                 0.127   0.357     1.253    0.290   0.538   
stats_for_length (x of gs)         0.073   0.271     0.827    0.193   0.439   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.706    0.080   0.282              0.658   
stats_for_recall                   0.553    0.066   0.256              0.444   
stats_for_f1-score                 0.620      NaN     NaN              0.527   
stats_for_f2-score                 0.578      NaN     NaN              0.473   
stats_for_NDCG                     0.691    0.067   0.259              0.575   
stats_for_M1                       0.644    0.229   0.479              0.555   
stats_for_M3                       0.844    0.131   0.362              0.685   
stats_for_M5                       0.844    0.131   0.362              0.685   
stats_for_position                 1.303    0.343   0.585              1.246   
stats_for_length (x of gs)         0.911    0.285   0.534              0.793   

                           average_doctor  
                                           
stats_for_precision                 0.701  
stats_for_recall                    0.454  
stats_for_f1-score                  0.548  
stats_for_f2-score                  0.487  
stats_for_NDCG                      0.599  
stats_for_M1                       

Now let us combine the individual results of the apps. We take the app results from different experiments and combine them. Our goal is to get the best results for each app and compare them.

In [31]:
bestResults = defaultdict(lambda : list(results.values())[0].copy())

for app in caseClassification['apps']:
    try:
        bestResults[getExpName(DiseaseType.ALL, FailureType.ANY, "best")][app] = results[
            getExpName(DiseaseType.ALL, FailureType.ANY, app)][app]
        bestResults[getExpName(DiseaseType.ALL, FailureType.SES_FAIL, "best")][app] = results[
            getExpName(DiseaseType.ALL, FailureType.ANY, app)][app]
        bestResults[getExpName(DiseaseType.ALL, FailureType.NO_DDX, "best")][app] = results[
            getExpName(DiseaseType.ALL, FailureType.ANY, app)][app]
    except Exception as e:
        display(getExpName(DiseaseType.ALL, FailureType.ANY, app))
        display(results[getExpName(DiseaseType.ALL, FailureType.ANY, app)])
        raise e

In [32]:
displayResults(bestResults,printNumCases=False)

Results for experiment `disease_type=all failure-type-ignored=any-error for app=best` is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.471    0.079   0.282   0.423    0.072   0.268   
stats_for_recall             0.555    0.086   0.293   0.538    0.085   0.292   
stats_for_f1-score           0.510      NaN     NaN   0.474      NaN     NaN   
stats_for_f2-score           0.536      NaN     NaN   0.510      NaN     NaN   
stats_for_NDCG               0.616    0.145   0.381   0.605    0.112   0.334   
stats_for_M1                 0.429    0.245   0.495   0.553    0.247   0.497   
stats_for_M3                 0.732    0.196   0.443   0.681    0.217   0.466   
stats_for_M5                 0.750    0.188   0.433   0.713    0.205   0.452   
stats_for_position           1.619    0.760   0.872   1.403    0.748   0.865   
stats_for_length (x of gs)   1.324    0.288   0.537   1.573    0.652   0.808   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.373    0.062   0.248   0.415  ...   
stats_for_recall             0.581    0.117   0.343   0.636  ...   
stats_for_f1-score           0.454      NaN     NaN   0.502  ...   
stats_for_f2-score           0.523      NaN     NaN   0.575  ...   
stats_for_NDCG               0.507    0.107   0.327   0.633  ...   
stats_for_M1                 0.231    0.178   0.422   0.513  ...   
stats_for_M3                 0.593    0.241   0.491   0.726  ...   
stats_for_M5                 0.676    0.219   0.468   0.744  ...   
stats_for_position           2.055    1.257   1.121   1.414  ...   
stats_for_length (x of gs)   1.753    0.471   0.686   1.743  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.141   0.375     0.801    0.069   0.262   
stats_for_recall                   0.050   0.223     0.512    0.054   0.233   
stats_for_f1-score                   NaN     NaN     0.625      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.552      NaN     NaN   
stats_for_NDCG                     0.108   0.329     0.703    0.074   0.272   
stats_for_M1                       0.239   0.489     0.696    0.211   0.460   
stats_for_M3                       0.220   0.469     0.839    0.135   0.367   
stats_for_M5                       0.220   0.469     0.839    0.135   0.367   
stats_for_position                 0.188   0.433     1.170    0.141   0.376   
stats_for_length (x of gs)         0.050   0.224     0.715    0.142   0.377   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.756    0.064   0.253              0.688   
stats_for_recall                   0.521    0.047   0.216              0.433   
stats_for_f1-score                 0.617      NaN     NaN              0.530   
stats_for_f2-score                 0.556      NaN     NaN              0.467   
stats_for_NDCG                     0.675    0.061   0.247              0.590   
stats_for_M1                       0.651    0.227   0.477              0.581   
stats_for_M3                       0.814    0.151   0.389              0.729   
stats_for_M5                       0.814    0.151   0.389              0.729   
stats_for_position                 1.229    0.233   0.483              1.256   
stats_for_length (x of gs)         0.776    0.164   0.404              0.720   

                           average_doctor  
                                           
stats_for_precision                 0.765  
stats_for_recall                    0.456  
stats_for_f1-score                  0.569  
stats_for_f2-score                  0.495  
stats_for_NDCG                      0.625  
stats_for_M1                       

Results for experiment `disease_type=all failure-type-ignored=session-failed for app=best` is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.471    0.079   0.282   0.423    0.072   0.268   
stats_for_recall             0.555    0.086   0.293   0.538    0.085   0.292   
stats_for_f1-score           0.510      NaN     NaN   0.474      NaN     NaN   
stats_for_f2-score           0.536      NaN     NaN   0.510      NaN     NaN   
stats_for_NDCG               0.616    0.145   0.381   0.605    0.112   0.334   
stats_for_M1                 0.429    0.245   0.495   0.553    0.247   0.497   
stats_for_M3                 0.732    0.196   0.443   0.681    0.217   0.466   
stats_for_M5                 0.750    0.188   0.433   0.713    0.205   0.452   
stats_for_position           1.619    0.760   0.872   1.403    0.748   0.865   
stats_for_length (x of gs)   1.324    0.288   0.537   1.573    0.652   0.808   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.373    0.062   0.248   0.415  ...   
stats_for_recall             0.581    0.117   0.343   0.636  ...   
stats_for_f1-score           0.454      NaN     NaN   0.502  ...   
stats_for_f2-score           0.523      NaN     NaN   0.575  ...   
stats_for_NDCG               0.507    0.107   0.327   0.633  ...   
stats_for_M1                 0.231    0.178   0.422   0.513  ...   
stats_for_M3                 0.593    0.241   0.491   0.726  ...   
stats_for_M5                 0.676    0.219   0.468   0.744  ...   
stats_for_position           2.055    1.257   1.121   1.414  ...   
stats_for_length (x of gs)   1.753    0.471   0.686   1.743  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.141   0.375     0.801    0.069   0.262   
stats_for_recall                   0.050   0.223     0.512    0.054   0.233   
stats_for_f1-score                   NaN     NaN     0.625      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.552      NaN     NaN   
stats_for_NDCG                     0.108   0.329     0.703    0.074   0.272   
stats_for_M1                       0.239   0.489     0.696    0.211   0.460   
stats_for_M3                       0.220   0.469     0.839    0.135   0.367   
stats_for_M5                       0.220   0.469     0.839    0.135   0.367   
stats_for_position                 0.188   0.433     1.170    0.141   0.376   
stats_for_length (x of gs)         0.050   0.224     0.715    0.142   0.377   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.756    0.064   0.253              0.688   
stats_for_recall                   0.521    0.047   0.216              0.433   
stats_for_f1-score                 0.617      NaN     NaN              0.530   
stats_for_f2-score                 0.556      NaN     NaN              0.467   
stats_for_NDCG                     0.675    0.061   0.247              0.590   
stats_for_M1                       0.651    0.227   0.477              0.581   
stats_for_M3                       0.814    0.151   0.389              0.729   
stats_for_M5                       0.814    0.151   0.389              0.729   
stats_for_position                 1.229    0.233   0.483              1.256   
stats_for_length (x of gs)         0.776    0.164   0.404              0.720   

                           average_doctor  
                                           
stats_for_precision                 0.765  
stats_for_recall                    0.456  
stats_for_f1-score                  0.569  
stats_for_f2-score                  0.495  
stats_for_NDCG                      0.625  
stats_for_M1                       

Results for experiment `disease_type=all failure-type-ignored=no-disease-found for app=best` is


Ada                  Ada_old                   \
                           average variance std Dev average variance std Dev   
stats_for_precision          0.471    0.079   0.282   0.423    0.072   0.268   
stats_for_recall             0.555    0.086   0.293   0.538    0.085   0.292   
stats_for_f1-score           0.510      NaN     NaN   0.474      NaN     NaN   
stats_for_f2-score           0.536      NaN     NaN   0.510      NaN     NaN   
stats_for_NDCG               0.616    0.145   0.381   0.605    0.112   0.334   
stats_for_M1                 0.429    0.245   0.495   0.553    0.247   0.497   
stats_for_M3                 0.732    0.196   0.443   0.681    0.217   0.466   
stats_for_M5                 0.750    0.188   0.433   0.713    0.205   0.452   
stats_for_position           1.619    0.760   0.872   1.403    0.748   0.865   
stats_for_length (x of gs)   1.324    0.288   0.537   1.573    0.652   0.808   

                              Avey                  Avey v2  ...  \
                           average variance std Dev average  ...   
stats_for_precision          0.373    0.062   0.248   0.415  ...   
stats_for_recall             0.581    0.117   0.343   0.636  ...   
stats_for_f1-score           0.454      NaN     NaN   0.502  ...   
stats_for_f2-score           0.523      NaN     NaN   0.575  ...   
stats_for_NDCG               0.507    0.107   0.327   0.633  ...   
stats_for_M1                 0.231    0.178   0.422   0.513  ...   
stats_for_M3                 0.593    0.241   0.491   0.726  ...   
stats_for_M5                 0.676    0.219   0.468   0.744  ...   
stats_for_position           2.055    1.257   1.121   1.414  ...   
stats_for_length (x of gs)   1.753    0.471   0.686   1.743  ...   

                           doctor_NJ_old         doctor_TH                   \
                                variance std Dev   average variance std Dev   
stats_for_precision                0.141   0.375     0.801    0.069   0.262   
stats_for_recall                   0.050   0.223     0.512    0.054   0.233   
stats_for_f1-score                   NaN     NaN     0.625      NaN     NaN   
stats_for_f2-score                   NaN     NaN     0.552      NaN     NaN   
stats_for_NDCG                     0.108   0.329     0.703    0.074   0.272   
stats_for_M1                       0.239   0.489     0.696    0.211   0.460   
stats_for_M3                       0.220   0.469     0.839    0.135   0.367   
stats_for_M5                       0.220   0.469     0.839    0.135   0.367   
stats_for_position                 0.188   0.433     1.170    0.141   0.376   
stats_for_length (x of gs)         0.050   0.224     0.715    0.142   0.377   

                           doctor_TH_old                  average_doctor_old  \
                                 average variance std Dev                      
stats_for_precision                0.756    0.064   0.253              0.688   
stats_for_recall                   0.521    0.047   0.216              0.433   
stats_for_f1-score                 0.617      NaN     NaN              0.530   
stats_for_f2-score                 0.556      NaN     NaN              0.467   
stats_for_NDCG                     0.675    0.061   0.247              0.590   
stats_for_M1                       0.651    0.227   0.477              0.581   
stats_for_M3                       0.814    0.151   0.389              0.729   
stats_for_M5                       0.814    0.151   0.389              0.729   
stats_for_position                 1.229    0.233   0.483              1.256   
stats_for_length (x of gs)         0.776    0.164   0.404              0.720   

                           average_doctor  
                                           
stats_for_precision                 0.765  
stats_for_recall                    0.456  
stats_for_f1-score                  0.569  
stats_for_f2-score                  0.495  
stats_for_NDCG                      0.625  
stats_for_M1                       